# Feature Engineering

In [1]:
import numpy as np
import pandas as pd

## FRED

In [20]:
# federal reserve economic data (fred)
from fredapi import Fred
fred = Fred(api_key_file='private/fred_api_key.txt')
test = fred.get_series('CUSR0000SETA01')

In [21]:
test.head()

1953-01-01    46.3
1953-02-01    46.8
1953-03-01    46.9
1953-04-01    47.3
1953-05-01    47.5
dtype: float64

In [24]:
fred.search('unemployment')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
UNRATE,UNRATE,2023-02-03,2023-02-03,Unemployment Rate,1948-01-01,2023-01-01,Monthly,M,Percent,%,Seasonally Adjusted,SA,2023-02-03 07:44:02-06:00,94,The unemployment rate represents the number of...
UNRATENSA,UNRATENSA,2023-02-03,2023-02-03,Unemployment Rate,1948-01-01,2023-01-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-02-03 07:43:02-06:00,57,The unemployment rate represents the number of...
CCSA,CCSA,2023-02-03,2023-02-03,Continued Claims (Insured Unemployment),1967-01-07,2023-01-21,"Weekly, Ending Saturday",W,Number,Number,Seasonally Adjusted,SA,2023-02-02 11:36:03-06:00,73,"Continued claims, also referred to as insured ..."
NROU,NROU,2023-02-03,2023-02-03,Noncyclical Rate of Unemployment,1949-01-01,2032-10-01,Quarterly,Q,Percent,%,Not Seasonally Adjusted,NSA,2022-05-26 13:45:04-05:00,67,"Starting with the July, 2021 report: An Update..."
UNEMPLOY,UNEMPLOY,2023-02-03,2023-02-03,Unemployment Level,1948-01-01,2023-01-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2023-02-03 07:44:05-06:00,69,The series comes from the 'Current Population ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WICH620UR,WICH620UR,2023-02-03,2023-02-03,"Unemployment Rate in Wichita, KS (MSA)",1990-01-01,2022-12-01,Monthly,M,Percent,%,Smoothed Seasonally Adjusted,SSA,2023-02-01 13:32:05-06:00,8,None
LEXI421URN,LEXI421URN,2023-02-03,2023-02-03,"Unemployment Rate in Lexington-Fayette, KY (MSA)",1990-01-01,2022-12-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-02-01 10:07:10-06:00,8,None
SAND706URN,SAND706URN,2023-02-03,2023-02-03,"Unemployment Rate in San Diego-Carlsbad, CA (MSA)",1990-01-01,2022-12-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2023-02-01 10:07:10-06:00,8,None


## uszipcode
- https://pypi.org/project/uszipcode/#the-data-point
- https://uszipcode.readthedocs.io/01-Usage-Example/index.html

In [2]:
# zipcode / census data
import uszipcode

In [15]:
from uszipcode import SearchEngine

# use simple search
sr = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.simple)

y = sr.by_zipcode(37209)
print(y)

SimpleZipcode(zipcode='37209', zipcode_type='STANDARD', major_city='Nashville', post_office_city='Nashville, TN', common_city_list=['Nashville'], county='Davidson County', state='TN', lat=36.19, lng=-86.95, timezone='America/Chicago', radius_in_miles=7.0, area_code_list='615', population=35457, population_density=976.0, land_area_in_sqmi=36.33, water_area_in_sqmi=1.1, housing_units=15648, occupied_housing_units=14130, median_home_value=170000, median_household_income=43625, bounds_west=-86.995828, bounds_east=-86.808173, bounds_north=36.209566, bounds_south=36.080921)


In [16]:
from uszipcode import SearchEngine

# use comprehensive search
sr = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive)

z = sr.by_zipcode(37209)
print(z)

ComprehensiveZipcode(zipcode='37209', zipcode_type='STANDARD', major_city='Nashville', post_office_city='Nashville, TN', common_city_list=['Nashville'], county='Davidson County', state='TN', lat=36.19, lng=-86.95, timezone='America/Chicago', radius_in_miles=7.0, area_code_list='615', population=35457, population_density=976.0, land_area_in_sqmi=36.33, water_area_in_sqmi=1.1, housing_units=15648, occupied_housing_units=14130, median_home_value=170000, median_household_income=43625, bounds_west=-86.995828, bounds_east=-86.808173, bounds_north=36.209566, bounds_south=36.080921, polygon=[[[-86.922643, 36.111294], [-86.922625, 36.111096], [-86.922614, 36.110974], [-86.922612, 36.110948], [-86.922582, 36.110603], [-86.922576, 36.110534], [-86.922573, 36.110502], [-86.922571, 36.110465], [-86.922564, 36.110384], [-86.922559, 36.110312], [-86.92255, 36.110221], [-86.922548, 36.110169], [-86.922537, 36.110022], [-86.922523, 36.109837], [-86.922373, 36.109872], [-86.922225, 36.109945], [-86.9221

In [17]:
z.source_of_earnings

[{'key': 'Data',
  'values': [{'x': 'Worked Full-time With Earnings', 'y': 12572},
   {'x': 'Worked Part-time With Earnings', 'y': 8789},
   {'x': 'No Earnings', 'y': 8720}]}]

In [19]:
z.to_dict()

{'zipcode': '37209',
 'zipcode_type': 'STANDARD',
 'major_city': 'Nashville',
 'post_office_city': 'Nashville, TN',
 'common_city_list': ['Nashville'],
 'county': 'Davidson County',
 'state': 'TN',
 'lat': 36.19,
 'lng': -86.95,
 'timezone': 'America/Chicago',
 'radius_in_miles': 7.0,
 'area_code_list': '615',
 'population': 35457,
 'population_density': 976.0,
 'land_area_in_sqmi': 36.33,
 'water_area_in_sqmi': 1.1,
 'housing_units': 15648,
 'occupied_housing_units': 14130,
 'median_home_value': 170000,
 'median_household_income': 43625,
 'bounds_west': -86.995828,
 'bounds_east': -86.808173,
 'bounds_north': 36.209566,
 'bounds_south': 36.080921,
 'polygon': [[[-86.922643, 36.111294],
   [-86.922625, 36.111096],
   [-86.922614, 36.110974],
   [-86.922612, 36.110948],
   [-86.922582, 36.110603],
   [-86.922576, 36.110534],
   [-86.922573, 36.110502],
   [-86.922571, 36.110465],
   [-86.922564, 36.110384],
   [-86.922559, 36.110312],
   [-86.92255, 36.110221],
   [-86.922548, 36.110169